<a href="https://colab.research.google.com/github/JuanjoRestrepo/Qatar-2022/blob/main/WorldCup_Qatar2022_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle
import time

# **Recolección de Datos**

In [ ]:
#all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')
all_tables = pd.read_html('https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

## **Obteniendo los grupos**

Como son 8 grupos (A -> H) y en el link las tablas de estos están ordenados cada 7 posiciones (desde la 12) haremos lo siguiente para obtener la tabla de cada grupo:
- 11 -> 7*8 + 12 = 67

In [ ]:
all_tables[12]
all_tables[19]
all_tables[26]
all_tables[61]

,Pos,Teamvte,Pld,W,D,L,GF,GA,GD,Pts,Qualification
0,1,Portugal,0,0,0,0,0,0,0,0,Advance to knockout stage
1,2,Ghana,0,0,0,0,0,0,0,0,Advance to knockout stage
2,3,Uruguay,0,0,0,0,0,0,0,0,NaN
3,4,South Korea,0,0,0,0,0,0,0,0,NaN


Como vemos el último grupo (H) está en la posición 61, por lo cual haremos un recorrido hasta ese rango que calculamos anteriorment y haremos unas modificaciones al formato de la tabla original que nos presenta el dataset.

Reemplazaremos el nombre de la columna 'Teamvte' por solo 'Team' pero hay una excepción en el primer grupo (A) y es que tiene un formato de 'Team.mw' en su columna de equipos, por lo cual haremos lo siguiente

In [ ]:
all_tables[12].columns[1]

'Team.mw-parser-output .navbar{display:inline;font-size:88%;font-weight:normal}.mw-parser-output .navbar-collapse{float:left;text-align:left}.mw-parser-output .navbar-boxtext{word-spacing:0}.mw-parser-output .navbar ul{display:inline-block;white-space:nowrap;line-height:inherit}.mw-parser-output .navbar-brackets::before{margin-right:-0.125em;content:"[ "}.mw-parser-output .navbar-brackets::after{margin-left:-0.125em;content:" ]"}.mw-parser-output .navbar li{word-spacing:-0.125em}.mw-parser-output .navbar a>span,.mw-parser-output .navbar a>abbr{text-decoration:inherit}.mw-parser-output .navbar-mini abbr{font-variant:small-caps;border-bottom:none;text-decoration:none;cursor:inherit}.mw-parser-output .navbar-ct-full{font-size:114%;margin:0 7em}.mw-parser-output .navbar-ct-mini{font-size:114%;margin:0 4em}vte'

Asignamos el valor numerico de las tablas a su grupo correspondiente

In [ ]:
for letter, i in zip(alphabet, range(12, 68, 7)):
  print(letter, i)

A 12
B 19
C 26
D 33
E 40
F 47
G 54
H 61


In [ ]:
dict_tables = {}
for letter, i in zip(alphabet, range(12, 68, 7)):
  df = all_tables[i]
  df.rename(columns={df.columns[1]: 'Team'}, inplace=True)
  df.pop('Qualification')
  dict_tables[f'Group {letter}'] = df

In [ ]:
dict_tables.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [ ]:
dict_tables['Group H']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Portugal,0,0,0,0,0,0,0,0
1,2,Ghana,0,0,0,0,0,0,0,0
2,3,Uruguay,0,0,0,0,0,0,0,0
3,4,South Korea,0,0,0,0,0,0,0,0


## **Exportamos nuestro Diccionario**
Abrimos un archivo para colocar el diccionario de las tablas en un archivo llamado 'output'

In [ ]:
with open('dict_table', 'wb') as output:
  pickle.dump(dict_tables, output)

## **Extraemos la data de todos los mundiales desde 1930 hasta 2018**

Haremos el Webscrapping para obtener estos datos con 'requests' y bs4

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

years = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 1974,
         1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014,
         2018]

def get_matches(year):
  if year == '2022':
    web =  f'https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'
  else:
    web = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup'

  response = requests.get(web)
  content = response.text #contenido html de la pagina
  soup = BeautifulSoup(content, 'lxml')

  matches = soup.find_all('div', class_='footballbox')

  home = []
  score = []
  away = []

  for game in matches:
      home.append(game.find('th', class_='fhome').get_text())
      score.append(game.find('th', class_='fscore').get_text())
      away.append(game.find('th', class_='faway').get_text())

  dict_football = {'home': home,
                  'score': score,
                  'away': away}

  df_football = pd.DataFrame(dict_football)
  df_football['year'] = year
  return df_football


Data Historica de todos los mundiales realizados

In [ ]:
fifa = [get_matches(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)
df_fifa.to_csv('fifa_worldcup_historical_data.csv', index=False)

Data del mundial Qatar 2022

In [ ]:
df_fixture = get_matches('2022')
df_fixture = df_fixture
df_fixture.to_csv('fifa_worldcup_fixture.csv', index=False)

In [ ]:
driver = web_driver()

web = 'https://en.wikipedia.org/wiki/1982_FIFA_World_Cup'

driver.get(web)
print

NameError: name 'web_driver' is not defined

In [ ]:
matches = driver.find_elements(by='xpath', value='//td[@align="right"]/..')


NameError: name 'driver' is not defined